#### Import standard python modules

In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

#### Import Azure ML SDK modules

In [3]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

#### Check Azure ML SDK version

In [4]:
print(azureml.core.VERSION)

1.16.0


#### Create Azure ML Workspace

In [33]:
AZ_SUBSCRIPTION_ID='3597c220-941e-4545-83df-b150daa004b7'
ws = Workspace.create(name='salary',
                      subscription_id=AZ_SUBSCRIPTION_ID, 
                      resource_group='webinar',
                      create_resource_group=True,
                      location='Southeast Asia'
                     ) 

Deploying AppInsights with name salaryinsightsb28fd0ca94.
Deployed AppInsights with name salaryinsightsb28fd0ca94. Took 5.12 seconds.
Deploying KeyVault with name salarykeyvaultc9345ce6bb.
Deploying StorageAccount with name salarystorage2e05e955583.
Deployed KeyVault with name salarykeyvaultc9345ce6bb. Took 20.72 seconds.
Deployed StorageAccount with name salarystorage2e05e955583. Took 25.03 seconds.
Deploying Workspace with name salary.
Deployed Workspace with name salary. Took 91.88 seconds.


#### Write configuration to local file

In [35]:
ws.write_config()

#### Create Azure ML Experiment

In [36]:
exp = Experiment(workspace=ws, name='salexp')

#### Start logging metrics

In [37]:
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

In [38]:
#### Load Dataset
sal = pd.read_csv('sal.csv',header=0, index_col=None)
X = sal[['x']]
y = sal['y']

#### Split the train and test data

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

#### Train the Model

In [40]:
lm = LinearRegression() 
lm.fit(X_train,y_train)

LinearRegression()

#### Freeze the model

In [41]:
filename = 'outputs/sal_model.pkl'
joblib.dump(lm, filename)

['outputs/sal_model.pkl']

#### Test the model

In [42]:
filename = 'outputs/sal_model.pkl'
loaded_model=joblib.load(filename)
y=loaded_model.predict([[30]])[0]
print(y)

158322.15447154472


#### Log metrics to Azure ML Experiment

In [43]:
run.log('Intercept :', lm.intercept_)
run.log('Slope :', lm.coef_[0])

#### End Azure ML Experiment

In [44]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

#### Get Portal URL

In [45]:
print(run.get_portal_url())

https://ml.azure.com/experiments/salexp/runs/87e6061a-c430-445d-8fc4-44a3c7d39260?wsid=/subscriptions/3597c220-941e-4545-83df-b150daa004b7/resourcegroups/webinar/workspaces/salary


#### Register the model

In [46]:
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model",
                       tags = {"key": "1"},
                       description = "Salary Prediction",
                       workspace = ws)

Registering model sal_model


#### Define Azure ML Deploymemt configuration

In [48]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Salary",  "method" : "sklearn"}, 
                                               description='Predict Stackoverflow Salary')

#### Create enviroment configuration file

In [50]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



#### Create Azure ML Scoring file

In [51]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Writing score.py


#### Deploy the model to Azure Container Instance

In [52]:
%%time
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="salenv.yml")

Wall time: 32.9 ms


C:\Users\Devaraj\anaconda3\envs\aml\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  This is separate from the ipykernel package so we can avoid doing imports until


#### Expose web service

In [ ]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='salary-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

C:\Users\Devaraj\anaconda3\envs\aml\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  """


Creating image
Running...............................................................
Succeeded
Image creation operation finished for image salary-svc:1, operation "Succeeded"
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................................................................................